In [22]:
import numpy as np
import pandas as pd

## Setting up a table with a linear integer index

Our assumption table has string columns, but we don't use these to identify anything. We use the `index` column which is a unique identifier for `[Sex, Underwriting]` combinations.

In [23]:

class MortalityTable:
    def __init__(self, select, ultimate):
        self.select = select
        self.ultimate = ultimate
    def __call__(self, linear_index, issue_age, duration): # duration starts at 0
        return np.where(
            duration < self.select.shape[-1],
            self.select[linear_index, issue_age-18, np.minimum(duration, self.select.shape[-1] - 1)],
            self.ultimate[linear_index, issue_age-18+duration],
            
        )
    def __repr__(self):
        return f"MortalityTable(select={self.select}, ultimate={self.ultimate})"
    
    @classmethod
    def from_filesystem(cls):
        with open('assumption_tables/2017_loaded_CSO_mortality_rates_select.npy', 'rb') as f:
            select = np.load(f)
        with open('assumption_tables/2017_loaded_CSO_mortality_rates_ultimate.npy', 'rb') as f:
            ultimate = np.load(f)
        return cls(select, ultimate)
    
mort = MortalityTable.from_filesystem()

## Join the assumption

This happens once.

In [37]:
mp = pd.read_csv('modelpoints_tables/2017_loaded_CSO_modelpoints.csv')
index_lookup = pd.read_csv('assumption_tables/2017_loaded_CSO_index.csv')[['index', 'Underwriting', 'Sex']]

In [42]:
mp.columns

Index(['Underwriting', 'Sex', 'IssueAge', 'Duration'], dtype='object')

In [38]:
%%timeit
mp.merge(index_lookup, on=['Underwriting', 'Sex'])

17.8 ms ± 2.96 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
class EnrichedModelPoints:
    def __init__(self, mp, index_lookup):
        self.enriched_mp = mp.merge(index_lookup, on=['Underwriting', 'Sex'])
        self.mort_index = self.enriched_mp['index'].values
        self.issue_age = self.enriched_mp['IssueAge'].values
        self.initial_duration = self.enriched_mp['Duration'].values

enriched_mp = EnrichedModelPoints(mp, index_lookup)

## Lookup values

In [46]:
%%timeit
mort(enriched_mp.mort_index, enriched_mp.issue_age, enriched_mp.initial_duration)

1.2 ms ± 95.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [47]:
mort(enriched_mp.mort_index, enriched_mp.issue_age, enriched_mp.initial_duration)

array([0.0003 , 0.00221, 0.00071, ..., 0.00101, 0.00046, 0.00049])